In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm

In [2]:
response = requests.get("https://en.wikipedia.org/wiki/Billboard_year-end_top_singles_of_1946")
soup = BeautifulSoup(response.text, "html.parser")

In [3]:
link_table = soup.find_all("tbody")[1]
links = link_table.find_all("a")[7:]

In [4]:
Wiki_ranking_links = ["/wiki/Billboard_year-end_top_singles_of_1946"]
for link in tqdm(links):
    Wiki_ranking_links.append(link.get("href"))

100%|██████████| 77/77 [00:00<00:00, 76458.67it/s]


In [28]:
df_all_rankings = pd.DataFrame()
year = 1946

for link in tqdm(Wiki_ranking_links):
    dfs = pd.read_html(f"https://en.wikipedia.org{link}")
    if year == 2012 or year == 2013:
        df_ranking = dfs[1]
    elif year == 2023:
        df_ranking = dfs[2]
    else:
        df_ranking = dfs[0]
    df_year_and_rank = pd.DataFrame(data={"year": [year for _ in range(len(df_ranking))], "rank": [i + 1 for i in range(len(df_ranking))]})
    df_ranking = pd.concat([df_year_and_rank, df_ranking.iloc[:,1], df_ranking.iloc[:,2]], axis=1)
    df_all_rankings = pd.concat([df_all_rankings, df_ranking], axis=0)
    year += 1

100%|██████████| 78/78 [00:46<00:00,  1.67it/s]


In [6]:
df_all_rankings.reset_index(drop=True, inplace=True)
df_all_rankings.head(5)

,year,rank,Title,Artist(s)
0,1946,1,"""Prisoner of Love""",Perry Como
1,1946,2,"""To Each His Own""",Eddy Howard
2,1946,3,"""The Gypsy""",The Ink Spots
3,1946,4,"""Five Minutes More""",Frank Sinatra
4,1946,5,"""Rumors Are Flying""",Frankie Carle


In [7]:
# "Stoned Soul Picnic”となっており、ダブルクォーテーションがおかしい
df_all_rankings.query('year == 1968 and rank == 17')

,year,rank,Title,Artist(s)
1404,1968,17,"""Stoned Soul Picnic”",The 5th Dimension


In [8]:
# 修正
df_all_rankings.iloc[1404, 2] = '"Stoned Soul Picnic"'

In [20]:
def title_modification(title: str, year: int) -> str:
    """
    レコードのA面とB面のタイトルを分離。\\
    また、タイトルにつけられている余分なダブルクォーテーションを取り除く。
    """
    if title.find("/") != -1:
        if title.find('" / "') != -1:
            idx = title.find('" / "')
            title1 = title[1:idx]
            title2 = title[idx+5:-1]
            return(f"{title1}|{title2}")
        elif title.find('"/"') != -1:
            idx = title.find('"/"')
            title1 = title[1:idx]
            title2 = title[idx+3:-1]
            return(f"{title1}|{title2}")
        else:
            # 1999年以降のタイトルに含まれるスラッシュは、そもそもスラッシュがタイトルの一部であるため分離しない。
            # 例えば"7/11"や"0 to 100 / The Catch Up"など。
            if year <= 1998:
                idx = title.find("/")
                title1 = title[1:idx]
                title2 = title[idx+1:-1]
                return(f"{title1}|{title2}")
    else:
        return(title[1:-1])

In [16]:
def artist_modification(artist: str) -> str:
    """
    "Artist1 & Artist2", "Artist1 featuring Artist2", "Artist1 with Artist2"のような場合に対し、先頭のアーティスト（Artist1）のみを抽出する。
    """
    # 基本的には先頭のアーティスト名義で曲が発表されていると考えてよい。
    # "A and B", "A & B"など、andや&自体がアーティスト名に含まれている場合があるが、この場ではその判定は行わない。
    # Genius APIで歌詞を検索する際にチェックする。
    idx_feat = artist.find(" featuring ")
    idx_and = artist.find(" and ")
    idx_and_sig = artist.find(" & ")
    idx_with = artist.find(" with ")
    idxs = [idx_feat, idx_and, idx_and_sig, idx_with]
    valid_idxs = [idx for idx in idxs if idx != -1]
    if valid_idxs == []:
        return artist
    else:
        min_idx = min(valid_idxs)
        artist = artist[:min_idx]
        return artist

In [22]:
r, c = df_all_rankings.shape
modified_titles = [""] * r
modified_artists = [""] * r

for i in tqdm(range(r)):
    year = df_all_rankings.iloc[i, 0]
    title = df_all_rankings.iloc[i, 2]
    artist = df_all_rankings.iloc[i, 3]
    modified_title = title_modification(title, year)
    modified_artist = artist_modification(artist)
    modified_titles[i] = modified_title
    modified_artists[i] = modified_artist

100%|██████████| 6989/6989 [00:00<00:00, 8280.94it/s]


In [30]:
df_final = df_all_rankings.copy(deep=True)
df_final["modified_title"] = modified_titles
df_final["modified_artist"] = modified_artists

In [34]:
df_final.rename(columns={"Title": "title", "Artist(s)": "artist"}, inplace=True)
df_final

,year,rank,title,artist,modified_title,modified_artist
0,1946,1,"""Prisoner of Love""",Perry Como,Prisoner of Love,Perry Como
1,1946,2,"""To Each His Own""",Eddy Howard,To Each His Own,Eddy Howard
2,1946,3,"""The Gypsy""",The Ink Spots,The Gypsy,The Ink Spots
3,1946,4,"""Five Minutes More""",Frank Sinatra,Five Minutes More,Frank Sinatra
4,1946,5,"""Rumors Are Flying""",Frankie Carle,Rumors Are Flying,Frankie Carle
...,...,...,...,...,...,...
95,2023,96,"""Bzrp Music Sessions, Vol. 53""",Bizarrap and Shakira,"Bzrp Music Sessions, Vol. 53",Bizarrap
96,2023,97,"""Meltdown""",Travis Scott featuring Drake,Meltdown,Travis Scott
97,2023,98,"""Put It on da Floor Again""",Latto featuring Cardi B,Put It on da Floor Again,Latto
98,2023,99,"""Bloody Mary""",Lady Gaga,Bloody Mary,Lady Gaga


In [ ]:
df_final.to_csv("data/ranking.csv", index=False)